# Music Recommendation System

In [1]:
import requests
import base64

In [2]:
#CLIENT_ID='e985e18e0d22454a879ac5fc0fb6d785' ----->You can use your own client ID
#CLIENT_SECRET='d552020c98b94e41a39a1017c14f381c'----->You can use your own client secret

client_credentials=f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [3]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [4]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [5]:
playlist_id = '37i9dQZF1DX0XUfTFmNBRM'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,"Satranga (From ""ANIMAL"")","Arijit Singh, Shreyas Puranik, Siddharth - Garima","Satranga (From ""ANIMAL"")",5mZX4EMwEyohNmVfLTDtXn,3yHyiUDJdz02FZ6jfUbsmY,88,2023-10-27,271169,False,https://open.spotify.com/track/3yHyiUDJdz02FZ6...,...,0.405,0,-5.810,1,0.0298,0.7350,0.000000,0.1890,0.534,89.480
1,Husn,Anuv Jain,Husn,2SarxpD48O1DOiDIccyZ0W,0TL0LFcwIBF5eX7arDIKxY,89,2023-12-01,217895,False,https://open.spotify.com/track/0TL0LFcwIBF5eX7...,...,0.376,5,-9.974,1,0.0298,0.6820,0.000000,0.1420,0.240,151.928
2,Sadqay,"Aashir Wajahat, NAYEL, Nehaal Naseem",Sadqay,6DCDw6OlKLmVlVrp9J6YWD,2lX53kXvYnvpV8Vqfe4S7s,71,2024-02-01,154455,False,https://open.spotify.com/track/2lX53kXvYnvpV8V...,...,0.535,4,-8.151,0,0.0454,0.1190,0.611000,0.1110,0.479,101.055
3,"O Maahi (From ""Dunki"")","Pritam, Arijit Singh, Irshad Kamil","O Maahi (From ""Dunki"")",011VDmbOf50dqxmLnkdsoC,0qQ4IdhjzNr0gJhdMTf2n3,82,2023-12-11,233453,False,https://open.spotify.com/track/0qQ4IdhjzNr0gJh...,...,0.669,1,-6.836,1,0.0349,0.5620,0.000000,0.1770,0.475,116.973
4,Saanware,"Akhil Sachdeva, Kartik Dev",Saanware,5na4ciNJa3xQdvcjyUsyy5,4LzjjNaJfnbPtJFYfMC8dc,63,2024-02-12,196193,False,https://open.spotify.com/track/4LzjjNaJfnbPtJF...,...,0.707,6,-6.299,1,0.0405,0.0516,0.000796,0.0893,0.512,176.166
5,Heeriye (feat. Arijit Singh),"Jasleen Royal, Arijit Singh, Harrykahanhai",Heeriye (feat. Arijit Singh),1wt2WZBZZ9GhM0AC61l7SS,5PUXKVVVQ74C3gl5vKy9Li,84,2023-07-25,194857,False,https://open.spotify.com/track/5PUXKVVVQ74C3gl...,...,0.494,6,-9.628,0,0.0256,0.5160,0.003840,0.0759,0.558,105.007
6,"Chaleya (From ""Jawan"")","Anirudh Ravichander, Arijit Singh, Shilpa Rao,...","Chaleya (From ""Jawan"")",7zAITOBN6eG4UBm4IapAik,4nc6XiUze2Yh7wFueGOPv7,82,2023-08-14,200374,False,https://open.spotify.com/track/4nc6XiUze2Yh7wF...,...,0.845,0,-5.280,1,0.0852,0.3930,0.010700,0.1310,0.439,95.047
7,Pehle Bhi Main,"Vishal Mishra, Raj Shekhar",ANIMAL,0a183xiCHiC1GQd8ou7WXO,7yDHHVKLbvDmVw1XXhDDIO,89,2023-11-24,250158,False,https://open.spotify.com/track/7yDHHVKLbvDmVw1...,...,0.406,10,-10.230,0,0.0353,0.7680,0.000002,0.1070,0.281,76.944
8,"Dil Se Dil Tak (From ""Bawaal"")","Akashdeep Sengupta, Laqshay Kapoor, Kausar Mun...","Dil Se Dil Tak (From ""Bawaal"")",1vaX8lnpErNywb6HwJBOsw,1kTEGyqUJFgstwdgJFLqz1,76,2023-07-14,301036,False,https://open.spotify.com/track/1kTEGyqUJFgstwd...,...,0.500,3,-6.154,1,0.0370,0.8540,0.000000,0.1660,0.232,111.927
9,"What Jhumka ? (From ""Rocky Aur Rani Kii Prem K...","Pritam, Arijit Singh, Jonita Gandhi, Ranveer S...","What Jhumka ? (From ""Rocky Aur Rani Kii Prem K...",4mrXgKhegKi7Ol4pdN4z3Q,4QyX8CBSjcoq4iMZuvifyF,68,2023-07-12,213611,False,https://open.spotify.com/track/4QyX8CBSjcoq4iM...,...,0.823,2,-5.455,1,0.1000,0.2350,0.000000,0.2070,0.479,107.953


In [50]:
music_df.isnull().sum()

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

# Generating Awesome Recommendations

We're going to employ two main approaches for generating music recommendations: content-based filtering and popularity-based filtering.

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

# Popularity-Based Filtering

Here, we recommend music based on their popularity scores. We'll even throw in a twist by weighting recent releases more heavily. 

In [7]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [8]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

# Content-Based Filtering

This method recommends music based on the similarity of their audio features. Using Spotipy, we'll fetch music data from Spotify and calculate similarity scores based on these features.

In [9]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

# Hybrid Approach

The hybrid approach combines the best of both worlds, merging content-based and popularity-based recommendations to provide you with the ultimate personalized experience

In [10]:
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Convert the dictionary to a DataFrame
    new_recommendation = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    # Concatenate the new DataFrame with the content-based recommendations
    hybrid_recommendations = pd.concat([content_based_rec, new_recommendation], ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations


# Putting It to the Test

To see the system in action, simply provide an input song name. The system will then generate recommendations based on this input, using our hybrid approach.


In [12]:
input_song_name = input("Enter song: ")
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
recommendations

Enter song: Arjan Vailly
Hybrid recommended songs for 'Arjan Vailly':


,Track Name,Artists,Album Name,Release Date,Popularity
0,"Besharam Rang (From ""Pathaan"")","Vishal-Shekhar, Shilpa Rao, Caralisa Monteiro,...","Besharam Rang (From ""Pathaan"")",2022-12-12,77.0
2,"Ishq Nachaawe - From ""Kho Gaye Hum Kahan""","Karan Kanchan, Rashmeet Kaur, Yashraj",Kho Gaye Hum Kahan (Original Motion Picture So...,2023-12-10,74.0
4,"Leke Prabhu Ka Naam (From ""Tiger 3"")","Pritam, Arijit Singh, Nikhita Gandhi, Amitabh ...","Leke Prabhu Ka Naam (From ""Tiger 3"")",2023-10-23,73.0
3,Sanak,"Badshah, Lakhwinder Wadali",3:00 AM Sessions,2023-02-07,71.0
1,"Tere Vaaste (From ""Zara Hatke Zara Bachke"")","Sachin-Jigar, Varun Jain, Shadab Faridi, Altam...","Tere Vaaste (From ""Zara Hatke Zara Bachke"") - ...",2023-05-22,66.0
